# Limit Order Book Simulator Demo
This notebook demonstrates the LOB simulator functionality.

In [ ]:
import sys
sys.path.append('../python')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lob.simulator import OrderBook, LOBSimulator, Side

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
# Create and initialize order book
book = OrderBook(tick_size=0.01)

# Add some orders
np.random.seed(42)
for i in range(10):
    book.add_order(Side.BUY, 100 - i * 0.1, np.random.randint(100, 500))
    book.add_order(Side.SELL, 100 + i * 0.1, np.random.randint(100, 500))

# Get snapshot
snapshot = book.get_snapshot()
print(f'Mid Price: {snapshot.mid_price:.2f}')
print(f'Spread: {snapshot.spread:.2f}')
print(f'Best Bid: {snapshot.bids[0] if snapshot.bids else None}')
print(f'Best Ask: {snapshot.asks[0] if snapshot.asks else None}')

In [ ]:
# Run simulation
sim = LOBSimulator(arrival_model='poisson')
trades = sim.run_simulation(duration=60.0, snapshot_interval=1.0)
snapshots = sim.get_snapshots_df()

print(f'Number of trades: {len(trades)}')
print(f'Number of snapshots: {len(snapshots)}')

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Mid price evolution
axes[0, 0].plot(snapshots['timestamp'], snapshots['mid_price'])
axes[0, 0].set_title('Mid Price Evolution')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Price')

# Spread over time
axes[0, 1].plot(snapshots['timestamp'], snapshots['spread'])
axes[0, 1].set_title('Bid-Ask Spread')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Spread')

# Depth
axes[1, 0].plot(snapshots['timestamp'], snapshots['bid_depth'], label='Bid Depth')
axes[1, 0].plot(snapshots['timestamp'], snapshots['ask_depth'], label='Ask Depth')
axes[1, 0].set_title('Order Book Depth')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Total Quantity')
axes[1, 0].legend()

# Trade sizes
if len(trades) > 0:
    axes[1, 1].hist(trades['quantity'], bins=30, edgecolor='black')
    axes[1, 1].set_title('Trade Size Distribution')
    axes[1, 1].set_xlabel('Trade Size')
    axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Market quality metrics
metrics = sim.analyze_market_quality()

print('Market Quality Metrics:')
print('-' * 30)
for key, value in metrics.items():
    print(f'{key}: {value:.4f}')